In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import collections as cl
from datetime import datetime
from math import radians, sin, cos, asin, sqrt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import warnings
from scipy.stats import norm

%matplotlib inline

In [2]:
# Restaurant_id — идентификационный номер ресторана / сети ресторанов;
# City — город, в котором находится ресторан;
# Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
# Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
# Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
# Price Range — диапазон цен в ресторане;
# Number of Reviews — количество отзывов о ресторане;
# Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
# URL_TA — URL страницы ресторана на TripAdvosor;
# ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [5]:
df = pd.read_csv('main_task.csv')

In [6]:
# 4.3
# сколько типов кухонь представлено
cuisins = []
for lst in df['Cuisine Style'].loc[df['Cuisine Style'].isna()!=True].str.replace('[','').str.replace(']','').str.split(', '):
    cuisins+=lst
len(set(cuisins))

125

In [7]:
# какая хухня представлена в наибольшем кол-ве ресторанов
c= cl.Counter()
for lst in df['Cuisine Style'].loc[df['Cuisine Style'].isna()!=True].str.replace('[','').str.replace(']','').str.split(', '):
    for item in lst:
        c[item]+=1
c.most_common(1)

[("'Vegetarian Friendly'", 11189)]

In [8]:
df['Cuisine Style'].fillna('other').str.replace('[','').str.replace(']','').str.split(', ').apply(len).mean()

2.6224

## Что не включил в паййплайн

In [1]:
# Для нормализации датафрейма через библиотеку мешает пазница в размерностях новой колонки и номрализуемого диапазона
#scaler = MinMaxScaler()
#scaler.fit_transform(df['cuisine_index'].loc[df.city == 'London'].values.reshape(-1,1))

#### Извращаться с логарифмированием и взятием квадратного корня тоже не буду, т.к. для RF это не поомжет. С этого места пропускаем¶

In [ ]:
# добавляем колонки с логарифмом и корнем
df['ranking_sqrt']=df['ranking'].apply(np.sqrt)

In [ ]:
dic_norm_city_ranking_sqrt = {}
for city in cities:
    dic_norm_ranking_params ={}
    max = df.loc[df.city==city]['ranking_sqrt'].max()
    min=df.loc[df.city==city]['ranking_sqrt'].min()
    dic_norm_ranking_params['max']=max
    dic_norm_ranking_params['min']=min
    dic_norm_city_ranking_sqrt[city]=dic_norm_ranking_params

In [ ]:
def mm_norm_ranking_in_city_sqrt(id):
    ct = df.loc[df.ID_TA==id]['city'].iloc[0]
    rank = df.loc[df.ID_TA==id]['ranking_sqrt'].iloc[0]
    max = dic_norm_city_ranking_sqrt[ct]['max']
    min = dic_norm_city_ranking_sqrt[ct]['min']
    rank_norm = (rank - min)/(max-min)
    return rank_norm

In [ ]:
df['ranking_mmnorm_sqrt']=df['ID_TA'].apply(mm_norm_ranking_in_city_sqrt)

In [ ]:
df['ranking_log']=df['ranking'].apply(np.log)

In [ ]:
dic_norm_city_ranking_log = {}
for city in cities:
    dic_norm_ranking_params ={}
    max = df.loc[df.city==city]['ranking_log'].max()
    min=df.loc[df.city==city]['ranking_log'].min()
    dic_norm_ranking_params['max']=max
    dic_norm_ranking_params['min']=min
    dic_norm_city_ranking_log[city]=dic_norm_ranking_params

In [ ]:
def mm_norm_ranking_in_city_log(id):
    ct = df.loc[df.ID_TA==id]['city'].iloc[0]
    rank = df.loc[df.ID_TA==id]['ranking_log'].iloc[0]
    max = dic_norm_city_ranking_log[ct]['max']
    min = dic_norm_city_ranking_log[ct]['min']
    rank_norm = (rank - min)/(max-min)
    return rank_norm

In [ ]:
df['ranking_mmnorm_log']=df['ID_TA'].apply(mm_norm_ranking_in_city_log)

In [ ]:
### Про выбросы

In [ ]:
df.number_of_reviews.loc[df.number_of_reviews<3].count()
# минимально два отзыва

In [ ]:
df['URL_TA'].loc[df.number_of_reviews>6000].iloc[1]

In [ ]:
df[['number_of_reviews', 'URL_TA']].loc[df.number_of_reviews>6000]

In [ ]:
df.iloc[19251]['URL_TA']

In [ ]:
df.number_of_reviews.hist(bins=100)

In [ ]:
df.number_of_reviews.apply(np.sqrt).hist(bins=50)

In [ ]:
df.number_of_reviews.apply(np.log).hist(bins=50)

##### Выбросы есть. Попробуем их найти

In [ ]:
df.number_of_reviews.quantile(0.99)

In [ ]:
# Кажется на глаз, что значения больше 1600-1800 являются выбросами

In [ ]:
# межквантильный интервал

In [ ]:
def out_iqr(val):
    q1 = val.quantile(0.25)
    q3 = val.quantile(0.75)
    print(q1, q3)
    iqr = q3 - q1
    print(iqr)
    upper_bound = q3 + (1.5*iqr)
    print(upper_bound)
    lower_bound = q1 - (1.5*iqr) 
    print(lower_bound)
    return np.where((val > upper_bound) | (val < lower_bound))[0]

In [ ]:
out_ranking_iqr=out_iqr(df.number_of_reviews)

In [ ]:
# Анализ IQR показал, что значения выше 274 - выбросы

In [2]:
# z-score на три отклонения

In [ ]:
def outliers_z_score(val, threshold=3):
    mean = np.mean(val)
    std= np.std(val)
    z_scores = [(item - mean) / std for item in val]
    return np.where(np.abs(z_scores) > threshold)[0]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    out = outliers_z_score(df.number_of_reviews)

In [ ]:
len(df.number_of_reviews.loc[df.number_of_reviews>1000])

In [ ]:
# анализ показал, что начиная прмиерно с 1000 - выбросы

In [ ]:
# поиск по распределению

In [ ]:
def estimate_gaussian(val):
    mu = np.mean(val, axis=0)
    sigma = np.cov(val.T)
    return mu, sigma
    
def get_gaussian(mu, sigma):
    distribution = norm(mu, sigma)
    return distribution

def get_probs(distribution, val):
    return distribution.pdf(val)

In [ ]:
mu, sigma = estimate_gaussian(df.number_of_reviews.dropna())
print(mu, sigma)
distribution = get_gaussian(mu, sigma)
probabilities = get_probs(distribution, df.number_of_reviews.dropna())

In [ ]:
out_dist = np.where(probabilities < 0.999)[0]
len(out_dist)

In [ ]:
mu, sigma = estimate_gaussian(df.number_of_reviews.apply(np.log).dropna())
print(mu, sigma)
distribution = get_gaussian(mu, sigma)
probabilities = get_probs(distribution, df.number_of_reviews.apply(np.log).dropna())

In [ ]:
out_dist = np.where(probabilities < 0.05)[0]
len(out_dist)

In [ ]:
mu, sigma = estimate_gaussian(df.number_of_reviews.apply(np.sqrt).dropna())
print(mu, sigma)
distribution = get_gaussian(mu, sigma)
probabilities = get_probs(distribution, df.number_of_reviews.apply(np.sqrt).dropna())

In [ ]:
out_dist = np.where(probabilities < 0.014)[0]
len(out_dist)

In [ ]:
# показывает какую-то дичь

In [ ]:
# DBSCAN

In [ ]:
a = np.array(df['number_of_reviews'].dropna().values).reshape(-1, 1)

In [ ]:
out_dbscan = DBSCAN(eps=0.5, min_samples=10).fit(a)

In [ ]:
len(np.where(out_dbscan.labels_ == -1)[0])

In [ ]:
# поиск пересечения
len(set(np.where(out_dbscan.labels_ == -1)[0]).intersection(set(out)).intersection(set(out_ranking_iqr)))

##### Пересечение дало всего 45 значений. RandomForest нечувствителен к выбросам, поэтому забьем